In [4]:
import json
from algosdk.v2client import indexer
import networkx as nx
import matplotlib.pyplot as plt

# Connect to the Algorand Indexer
indexer_address = "https://mainnet-idx.algonode.cloud"
indexer_client = indexer.IndexerClient("", indexer_address)

In [5]:
def get_current_round():
    current_round = indexer_client.health().get("round")
    data = current_round
    return data



In [9]:

def fetch_transactions(min_round, max_round, limit=1000):
    transactions = []
    next_token = None
    while True:
        try:
            response = indexer_client.search_transactions(min_round=min_round, max_round=max_round, limit=limit, next_page=next_token)
            transactions_batch = response.get('transactions', [])
            transactions.extend(transactions_batch)
            next_token = response.get('next-token', None)
            if next_token is None or not transactions_batch:
                break
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    return transactions

# Fetch transactions from round x to the current round
# 60 / 3.6 = 16.6666 transactions per minute based on the 3.6s block time we round down to 15
# 15 * 60 * 24 = 21600 transactions per day
transactions = fetch_transactions(get_current_round()-5, get_current_round())
len(transactions)

112

In [10]:
import networkx as nx

def build_network(transactions):
    G = nx.DiGraph()  # Directed graph to capture the direction of transactions

    for txn in transactions:
        sender = txn.get('sender')
        receiver = txn.get('payment-transaction', {}).get('receiver')

        if sender and receiver:
            if G.has_edge(sender, receiver):
                G[sender][receiver]['weight'] += 1
            else:
                G.add_edge(sender, receiver, weight=1)

    return G

G = build_network(transactions)


In [11]:
# Compute centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

# Print the top 5 nodes for each centrality measure
print("Top 5 nodes by degree centrality:")
print(sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5])

print("\nTop 5 nodes by betweenness centrality:")
print(sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:5])

print("\nTop 5 nodes by closeness centrality:")
print(sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:5])

print("\nTop 5 nodes by eigenvector centrality:")
print(sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:5])


Top 5 nodes by degree centrality:
[('YHIYCCBE3Y72Z5DIHQFBBPL4PN2PJ7OWXEE53XYPAR7EETR5GQV4UHFHOE', 0.5555555555555556), ('5SFNTQF5WA7GWS5RQEDZRNQ22GTAPBTLGIQDOBKOSWVT7YOOVKHVMC4Q4U', 0.16666666666666666), ('HILOLPDRG7G3SMH3FH3BTK6CUR57YHOY5H752PCDVVDN42SJSO5B53M3WE', 0.05555555555555555), ('QYXDGS2XJJT7QNR6EJ2YHNZFONU6ROFM6BKTBNVT63ZXQ5OC6IYSPNDJ4U', 0.05555555555555555), ('H6BWVBLLPFF3XF7EG37AKEPUEXAV5TD2NWS6SOB4CCKXDO56A6QAPZLYPQ', 0.05555555555555555)]

Top 5 nodes by betweenness centrality:
[('5SFNTQF5WA7GWS5RQEDZRNQ22GTAPBTLGIQDOBKOSWVT7YOOVKHVMC4Q4U', 0.0), ('HILOLPDRG7G3SMH3FH3BTK6CUR57YHOY5H752PCDVVDN42SJSO5B53M3WE', 0.0), ('QYXDGS2XJJT7QNR6EJ2YHNZFONU6ROFM6BKTBNVT63ZXQ5OC6IYSPNDJ4U', 0.0), ('H6BWVBLLPFF3XF7EG37AKEPUEXAV5TD2NWS6SOB4CCKXDO56A6QAPZLYPQ', 0.0), ('CLNJRWDFUZOPDPFDG6Y4KCCGLLOKS7UWGEFKFMTU36UHSV4ZJ2LWBM6C2Y', 0.0)]

Top 5 nodes by closeness centrality:
[('YHIYCCBE3Y72Z5DIHQFBBPL4PN2PJ7OWXEE53XYPAR7EETR5GQV4UHFHOE', 0.5555555555555556), ('HILOLPDRG7G3SMH3FH3BTK6CUR57YH